In [1]:
# Social Media Mining - Caso Garlasco
# Obiettivo: raccogliere e salvare commenti da video YouTube relativi alla riapertura del caso Chiara Poggi (Garlasco), pubblicati tra marzo 2024 e luglio 2025.
from googleapiclient.discovery import build
from datetime import datetime
from collections import defaultdict
import json
import os
import random
import re
from collections import defaultdict

from dotenv import load_dotenv

In [2]:
# Rimuove una variabile specifica
#os.environ.pop("yt_key", None)  

In [3]:
# Configurazione dell'accesso alle API di YouTube
load_dotenv()
DEVELOPER_KEY = os.environ.get('yt_key')
if not DEVELOPER_KEY:
    raise ValueError("Chiave API non trovata")
youtube = build("youtube", "v3", developerKey=DEVELOPER_KEY)

In [4]:
# Canali target da cui raccogliere i video
channels = [
    "Fanpage.it",
    "La Repubblica",
    "FABRIZIO CORONA",
    "Gianmarco Zagato",
    "La7 Attualità",
    "Rai",
    "Bugalalla Crime",
    "DarkSide - Storia Segreta d'Italia"
]

# Intervallo temporale per i video (due settimana prima dalla riapertura del caso fino ai giorni odierni)
begin_date = datetime(2025, 2, 25)
end_date = datetime(2025, 7, 14)

In [5]:
# Ottieni ID canale da nome
def getIDfromName(name):
    request = youtube.search().list(
        part="snippet",
        q=name,
        type="channel",
        maxResults=1
    )
    response = request.execute()
    return response['items'][0]['id']['channelId']

# Ottieni playlist upload del canale
def getChannelPlaylist(channel_id):
    request = youtube.channels().list(
        part="contentDetails",
        id=channel_id
    )
    response = request.execute()
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

# Estrai video dalla playlist e filtra quelli relativi al caso Garlasco
def get_videos_from_channel(playlist_id, channel_name):
    video_data = []
    next_page_token = None

    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response['items']:
            try:
                snippet = item['snippet']
                video_date = datetime.strptime(snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ")
                title = snippet['title'].lower()
                description = snippet.get('description', '').lower()
                combined_text = title + description
                video_id = snippet['resourceId']['videoId']

                if begin_date <= video_date <= end_date and "garlasco" in combined_text:
                    video_req = youtube.videos().list(
                        part="statistics",
                        id=video_id
                    )
                    video_res = video_req.execute()

                    if not video_res['items']:
                        continue  

                    stats = video_res['items'][0].get('statistics', {})
                    if 'commentCount' not in stats:
                        continue  
                    video_entry = {
                        "video_id": video_id,
                        "published_at": video_date.strftime("%Y-%m-%d"),
                        "title": snippet['title'],
                        "channel": channel_name
                    }
                    video_data.append(video_entry)
            except Exception as e:
                print(f"[Errore parsing video]: {e}")
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    return video_data

# Funzione principale che unisce i risultati e li salva in un unico file JSON
def get_videos_json(channels, output_path="videos.json"):
    all_videos = []
    for channel in channels:
        try:
            print(f"\n>>> Ricerca nel canale: {channel}")
            channel_id = getIDfromName(channel)
            playlist_id = getChannelPlaylist(channel_id)
            videos = get_videos_from_channel(playlist_id, channel)
            all_videos.extend(videos)
        except Exception as e:
            print(f"[Errore canale {channel}]: {e}")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(all_videos, f, ensure_ascii=False, indent=2)
    print(f"\nSalvati {len(all_videos)} video in '{output_path}'")

In [6]:
# Avvio raccolta video
get_videos_json(channels)


>>> Ricerca nel canale: Fanpage.it

>>> Ricerca nel canale: La Repubblica

>>> Ricerca nel canale: FABRIZIO CORONA

>>> Ricerca nel canale: Gianmarco Zagato

>>> Ricerca nel canale: La7 Attualità

>>> Ricerca nel canale: Rai

>>> Ricerca nel canale: Bugalalla Crime

>>> Ricerca nel canale: DarkSide - Storia Segreta d'Italia

Salvati 352 video in 'videos.json'


In [7]:
# Crea struttura Comment
class Comment:
    def __init__(self, id, video_id, content, author, date, likes, reply_to_id=None):
        self.id = id
        self.video_id = video_id
        self.content = content
        self.author = author
        self.date = date
        self.likes = int(likes)
        self.reply_to_id = reply_to_id

    def __repr__(self):
        return f"<Comment by {self.author} on {self.date.strftime('%Y-%m-%d')}>"

In [8]:
# Estrae e struttura commenti da risposta API
def get_comments_from_response(items):
    comments = []
    for item in items:
        try:
            top = item["snippet"]["topLevelComment"]
            main = Comment(
                id=top["id"],
                video_id=item["snippet"]["videoId"],
                content=top["snippet"]["textDisplay"],
                author=top["snippet"]["authorDisplayName"],
                date=datetime.strptime(top["snippet"]["publishedAt"], "%Y-%m-%dT%H:%M:%SZ"),
                likes=top["snippet"]["likeCount"],
                reply_to_id=None
            )
            comments.append(main)

            if "replies" in item:
                for reply in item["replies"]["comments"]:
                    comments.append(Comment(
                        id=reply["id"],
                        video_id=item["snippet"]["videoId"],
                        content=reply["snippet"]["textDisplay"],
                        author=reply["snippet"]["authorDisplayName"],
                        date=datetime.strptime(reply["snippet"]["publishedAt"], "%Y-%m-%dT%H:%M:%SZ"),
                        likes=reply["snippet"]["likeCount"],
                        reply_to_id=top["id"]
                    ))
        except Exception as e:
            print(f"[Errore parsing commento]: {e}")
            continue
    return comments

# Estrae tutti i commenti da un singolo video
def get_comments_one_vid(video_id):
    comments = []
    try:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            textFormat="plainText",
            maxResults=100
        )
        response = request.execute()
        comments.extend(get_comments_from_response(response["items"]))
        next_token = response.get("nextPageToken")
        while next_token:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
                textFormat="plainText",
                maxResults=100,
                pageToken=next_token
            )
            response = request.execute()
            comments.extend(get_comments_from_response(response["items"]))
            next_token = response.get("nextPageToken")
    except Exception as e:
        print(f"[Errore] Estrazione commenti video {video_id}: {e}")
    return comments

# Carica gli ID video da un file JSON e raccoglie tutti i commenti in un file JSON unico
def get_all_comments(video_json_path, output_json_path="comments.json"):
    all_comments = []

    with open(video_json_path, "r", encoding="utf-8") as f:
        videos = json.load(f)
        video_ids = [video["video_id"] for video in videos]

    for idx, vid in enumerate(video_ids):
        print(f"[{idx+1}/{len(video_ids)}] Estrazione commenti per video ID: {vid}")
        comments = get_comments_one_vid(vid)
        for c in comments:
            all_comments.append({
                "comment_id": c.id,
                "video_id": c.video_id,
                "author": c.author,
                "content": c.content,
                "date": c.date.strftime("%Y-%m-%d"),
                "likes": c.likes,
                "reply_to_id": c.reply_to_id
            })

    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(all_comments, f, ensure_ascii=False, indent=2)

    print(f"\nSalvati {len(all_comments)} commenti in '{output_json_path}'")

In [9]:
# Estrazione commenti
get_all_comments("videos.json")

[1/352] Estrazione commenti per video ID: 02Tc4ZF227w
[2/352] Estrazione commenti per video ID: Ccf0jkSqBpk
[3/352] Estrazione commenti per video ID: 9sh52chcvu0
[4/352] Estrazione commenti per video ID: tFDL9Mx16jc
[5/352] Estrazione commenti per video ID: e2joMDbWfbQ
[6/352] Estrazione commenti per video ID: xrnTgGRJHz0
[7/352] Estrazione commenti per video ID: hvai9r6gBDc
[8/352] Estrazione commenti per video ID: 0E2znss1aPI
[9/352] Estrazione commenti per video ID: oaS-KJ0mz20
[10/352] Estrazione commenti per video ID: hMD5rLYDUrk
[11/352] Estrazione commenti per video ID: 9jNySIytgp4
[12/352] Estrazione commenti per video ID: -tIRGBdPabA
[13/352] Estrazione commenti per video ID: MkJULnHa9Ug
[14/352] Estrazione commenti per video ID: rGQlVoynlWo
[15/352] Estrazione commenti per video ID: jcym9wXec3c
[16/352] Estrazione commenti per video ID: ibyXAAz-7eg
[17/352] Estrazione commenti per video ID: s-HdzEdrcQQ
[18/352] Estrazione commenti per video ID: PK-Q-YpAASU
[19/352] Estrazione

In [10]:
# Funzione per arricchire i video con statistiche di visualizzazione e like
def get_video_views(input_json="garlasco_videos.json", output_json="garlasco_videos_with_stats.json"):
    """
    Carica il file JSON con i dati dei video, aggiunge numero di like e visualizzazioni e salva il nuovo JSON.
    """
    import json
    from time import sleep

    with open(input_json, "r", encoding="utf-8") as f:
        videos = json.load(f)

    enriched = []
    for i, video in enumerate(videos):
        video_id = video["video_id"]
        try:
            print(f"[{i+1}/{len(videos)}] Prendo statistiche per video ID: {video_id}")
            request = youtube.videos().list(
                part="statistics",
                id=video_id
            )
            response = request.execute()
            stats = response["items"][0]["statistics"] if response["items"] else {}
            video["likeCount"] = int(stats.get("likeCount", 0))
            video["viewCount"] = int(stats.get("viewCount", 0))
            enriched.append(video)
            sleep(0.1)  # Leggero delay per evitare throttling
        except Exception as e:
            print(f"Errore su video {video_id}: {e}")
    
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(enriched, f, ensure_ascii=False, indent=2)
    print(f"Salvato file aggiornato in '{output_json}'")


In [11]:
get_video_views(
    input_json="videos.json",
    output_json="videos_with_views.json"
)

[1/352] Prendo statistiche per video ID: 02Tc4ZF227w
[2/352] Prendo statistiche per video ID: Ccf0jkSqBpk
[3/352] Prendo statistiche per video ID: 9sh52chcvu0
[4/352] Prendo statistiche per video ID: tFDL9Mx16jc
[5/352] Prendo statistiche per video ID: e2joMDbWfbQ
[6/352] Prendo statistiche per video ID: xrnTgGRJHz0
[7/352] Prendo statistiche per video ID: hvai9r6gBDc
[8/352] Prendo statistiche per video ID: 0E2znss1aPI
[9/352] Prendo statistiche per video ID: oaS-KJ0mz20
[10/352] Prendo statistiche per video ID: hMD5rLYDUrk
[11/352] Prendo statistiche per video ID: 9jNySIytgp4
[12/352] Prendo statistiche per video ID: -tIRGBdPabA
[13/352] Prendo statistiche per video ID: MkJULnHa9Ug
[14/352] Prendo statistiche per video ID: rGQlVoynlWo
[15/352] Prendo statistiche per video ID: jcym9wXec3c
[16/352] Prendo statistiche per video ID: ibyXAAz-7eg
[17/352] Prendo statistiche per video ID: s-HdzEdrcQQ
[18/352] Prendo statistiche per video ID: PK-Q-YpAASU
[19/352] Prendo statistiche per video

In [16]:
soggetti = {
    "stefania": ["stefania", "stefania cappa"],
    "paola": ["paola", "paola cappa"],
    "andrea": ["andrea", "andrea sempio", "sempio"],
    "alberto": ["alberto", "alberto stasi", "stasi"],
    "marco": ["marco", "marco poggi"],
    "stefania e paola": ["stefania cappa e paola cappa", "stefania e paola", "gemelle cappa", "gemelle", "sorelle"],
}

def pulisci_testo(testo):
    testo = re.sub(r"http\S+", "", testo)
    testo = re.sub(r"@\w+", "", testo)
    testo = re.sub(r"[^a-zA-Zàèéìòùç0-9\s']", " ", testo)
    testo = re.sub(r"\s+", " ", testo)
    return testo.strip().lower()

In [17]:
def commenti_distribuiti(commenti_input_path, commenti_output_path, max_commenti=80000):
    with open(commenti_input_path, "r", encoding="utf-8") as f:
        all_comments = json.load(f)

    valid_comments = []
    for c in all_comments:
        if "date" in c and c["date"] and c.get("content", "").strip():
            try:
                c["parsed_date"] = datetime.fromisoformat(c["date"])
                if begin_date <= c["parsed_date"] <= end_date:
                    valid_comments.append(c)
            except Exception:
                continue

    if not valid_comments:
        print("Nessun commento nel range di date.")
        return

    gruppi = defaultdict(list)
    for c in valid_comments:
        settimana = c["parsed_date"].strftime("%Y-%W")
        gruppi[settimana].append(c)

    commenti_per_settimana = max(1, max_commenti // len(gruppi))

    selezionati = []
    for settimana, commenti in gruppi.items():
        n = min(len(commenti), commenti_per_settimana)
        selezionati.extend(random.sample(commenti, n))

    random.shuffle(selezionati)
    print(f"[FASE 1] Commenti selezionati da {len(gruppi)} settimane → {len(selezionati)} totali")

    for c in selezionati:
        c.pop("parsed_date", None)

    with open(commenti_output_path, "w", encoding="utf-8") as f:
        json.dump(selezionati, f, ensure_ascii=False, indent=2)

    print(f"[FASE 1] Salvati {len(selezionati)} commenti distribuiti in '{commenti_output_path}'")


In [18]:
def commenti_con_soggetti(commenti_input_path, commenti_output_path, max_commenti_finali=5000):
    with open(commenti_input_path, "r", encoding="utf-8") as f:
        all_comments = json.load(f)

    associabili = []
    for c in all_comments:
        if "content" in c:
            testo_pulito = pulisci_testo(c["content"])
            for varianti in soggetti.values():
                if any(v in testo_pulito for v in varianti):
                    associabili.append(c)
                    break

    print(f"[FASE 2] Trovati {len(associabili)} commenti associabili a soggetti")

    if len(associabili) > max_commenti_finali:
        associabili = random.sample(associabili, max_commenti_finali)
        print(f"[FASE 2] Ridotti a {max_commenti_finali} commenti selezionati casualmente")

    with open(commenti_output_path, "w", encoding="utf-8") as f:
        json.dump(associabili, f, ensure_ascii=False, indent=2)

    print(f"[FASE 2] Salvati {len(associabili)} commenti associabili in '{commenti_output_path}'")

In [19]:
commenti_distribuiti(
    commenti_input_path="comments.json",
    commenti_output_path="commenti_distribuiti.json",
    max_commenti=80000
)

commenti_con_soggetti(
    commenti_input_path="commenti_distribuiti.json",
    commenti_output_path="commenti_associabili.json",
    max_commenti_finali=14718
)

[FASE 1] Commenti selezionati da 19 settimane → 70214 totali
[FASE 1] Salvati 70214 commenti distribuiti in 'commenti_distribuiti.json'
[FASE 2] Trovati 14737 commenti associabili a soggetti
[FASE 2] Ridotti a 14718 commenti selezionati casualmente
[FASE 2] Salvati 14718 commenti associabili in 'commenti_associabili.json'
